**Researching Arbitrage Opportunities with Closed-End Fund ETFs**
__________

***Using:*** PHYS (physical gold etf) and GLD (spot gold etf) as a hedge against the NAV of PHYS

In [7]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.api import OLS
import yfinance as yf
import plotly.graph_objects as go

In [2]:
phys = pd.read_excel('excel_data/PHYS.xlsx')
# Set the index to the date
phys = phys.set_index('Date')
phys = phys.drop(columns=['Close'])
phys.head()

,NAV
Date,
2023-01-20,15.2667
2023-01-19,15.3157
2023-01-18,15.0929
2023-01-17,15.1294
2023-01-16,15.1877


In [3]:
# Join the phys dataframe with data from yfinance on the close for each day
phys_closes = yf.download('PHYS', start=min(phys.index), end=max(phys.index))
phys_closes.index = pd.to_datetime(phys_closes.index).date
phys.index = pd.to_datetime(phys.index).date
# Join the two dataframes
phys = phys.join(phys_closes['Close'], how='inner')
phys.head()

[*********************100%***********************]  1 of 1 completed


,NAV,Close
2023-01-19,15.3157,14.96
2023-01-18,15.0929,14.74
2023-01-17,15.1294,14.79
2023-01-13,15.2216,14.86
2023-01-12,15.0383,14.73


In [4]:
# plot the close and nav
fig = go.Figure()
fig.add_trace(go.Scatter(x=phys.index, y=phys['Close'], name='Close'))
fig.add_trace(go.Scatter(x=phys.index, y=phys['NAV'], name='NAV'))
fig.update_layout(title='PHYS Close and NAV', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [6]:
# Plot the spread
spread=phys['Close']-phys['NAV']
fig = go.Figure()
fig.add_trace(go.Scatter(x=phys.index, y=spread, name='Spread'))
fig.update_layout(title='PHYS Spread', xaxis_title='Date', yaxis_title='Spread')
fig.show()

In [11]:
# Calculating the mean-reverting nature of the daily spread itself
# First, remove nans from the spread
spread0 = spread.dropna()
adf_results = adfuller(spread0)
print('ADF Statistic: %f' % adf_results[0])
print('p-value: %f' % adf_results[1])

ADF Statistic: -3.654653
p-value: 0.004798


We can reject the null hypothesis that the spread is non-stationary on the grounds that the p-value of the null is $<$ 5% (@ 4.7%).

However, trading larger-scale mean-reversions on a daily timeframe is not what we are interested in. For the remainder of the research, we will be exploring the stationarity (and ability to profit off mean-reversions) of:


1) The rolling mean of the Price - NAV spread on different daily timeframes (and std bands built off of the mean)

2) Intraday deviations off of the rolling mean Price - NAV spread

3) **Intraday deviations off of the theoretical Price given the previous day Price/NAV premium/discount (i.e. is the premium/discount ratio on $t_0$ predictive of the premium/discount ratio on $t_1$)**


From here on, research will be moved to QuantConnect and reports will be linked in this notebook. The yfinance intraday data availability on low-volume tickers is not sufficient for conducting confident research.

In [28]:
# Helpers:
# calculate_expected_profit()
# calculate_theo()

def calculate_theo(D, P):
    """
    Method for calculating the theoretical price of PHYS given its NAV and a historical
    ratio for extrapolating the price of PHYS from its per-share NAV.

    Formula: theo = DP, where

    :param D: The percent change in the underlying (GLD) since the last NAV timestamp
    :param P: Price of PHYS at the last NAV timestamp
    :returns theo: The theoretical price of PHYS
    """

    return D * P



def calculate_expected_profit(theo, borrow_rate, sofr, settle_days, p, qty_phys, qty_hedge, p_hedge):
    """
    Method for calculating the expected profit of a trade given the theoretical price, 
    the actual current price, the 

    E(P) = | theo - p | - C_hedge - C_settlement, where
    | theo - p |: the value of the mean reversion
    C_hedge: the cost of the hedge
    C_settlement: the cost of the settlement
    """

    C_hedge = (borrow_rate / 365) * (qty_hedge * p_hedge)
    C_settlement = (settle_days * (sofr / 365)) * (qty_hedge * p_hedge + qty_phys * p)

    return qty_phys * (abs(theo - p)) - C_hedge - C_settlement

print(calculate_expected_profit(theo=15, p=14.95, borrow_rate=0.003, sofr=0.05, settle_days=2, p_hedge=15, qty_phys=100, qty_hedge=100))

41.67123287671304


In [79]:
def calculate_halflife(phys_df):
    """
    Method for calculating the half-life of the spread between the theoretical price of PHYS
    and its actual price.

    :param phys_df: The dataframe containing the price of PHYS
    :param theo_df: The dataframe containing the theoretical price of PHYS
    :returns halflife: The half-life of the spread between the theoretical price of PHYS and
    its actual price.
    """

    spread = phys_df['Adj Close'] - phys_df['Theo']
    prevspread = spread.shift()
    dz = spread - prevspread
    dz = dz[1:]
    prevspread = prevspread[1:]

    model = OLS(dz, prevspread - np.mean(prevspread))
    res = model.fit()
    theta = res.params[0]
    halflife = -np.log(2) / theta
    return halflife

In [87]:
phys_daily = yf.download('PHYS', start='2023-05-01', end='2023-05-10', interval='1d')
gld_daily = yf.download('GLD', start='2023-05-01', end='2023-05-10', interval='1d')
# convert the Datetime index to dates
phys_daily.index = pd.to_datetime(phys_daily.index).date
gld_daily.index = pd.to_datetime(gld_daily.index).date

phys_iday = yf.download('PHYS', start='2023-05-02', end='2023-05-09', interval='1h')
gld_iday = yf.download('GLD', start='2023-05-02', end='2023-05-09', interval='1h')

phys_iday['Theo'] = 0

# Join the GLD data into the PHYS df for iday
phys_iday = phys_iday.join(gld_iday['Close'], how='inner', lsuffix='_phys', rsuffix='_gld')

phys_iday.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close_phys,Adj Close,Volume,Theo,Close_gld
Datetime,,,,,,,,
2023-05-02 09:30:00-04:00,15.5300,15.645,15.510,15.6400,15.6400,609754,0,186.122498
2023-05-02 10:30:00-04:00,15.6400,15.700,15.640,15.6650,15.6650,1525152,0,186.869995
2023-05-02 11:30:00-04:00,15.6700,15.720,15.650,15.7037,15.7037,967649,0,187.389999
2023-05-02 12:30:00-04:00,15.7035,15.710,15.660,15.6700,15.6700,314427,0,187.149994
2023-05-02 13:30:00-04:00,15.6699,15.700,15.655,15.7000,15.7000,166691,0,187.250000


In [88]:
theos = []
for index, row in phys_iday.iterrows():
    # :param D: The percent change in the underlying (GLD) since the last NAV timestamp
    # :param P: Price of PHYS at the last NAV timestamp
    date = index.date()
    # get the close price of GLD for that date
    gld_close = gld_daily.loc[date]['Adj Close']
    phys_close = phys_daily.loc[date]['Adj Close']
    D = row['Close_gld'] / gld_close
    P = phys_close
    theos.append(calculate_theo(D, P))

phys_iday['Theo'] = theos

In [89]:
calculate_halflife(phys_iday)

1.2862989388569088

In [101]:
# for use in quantconnect, we have to rig the data into a dictionary then re-upload to quantconnect.
import datetime
# make a new df of phys that only contains data past 2018
phys2 = phys.loc[:datetime.date(2018, 1, 1)]

# add in the new gpt data from ycharts


In [104]:
import datetime

data = [
    ['June 05, 2023', 15.52],
    ['June 02, 2023', 15.41],
    ['June 01, 2023', 15.65],
    ['May 31, 2023', 15.53],
    ['May 30, 2023', 15.50],
    ['May 26, 2023', 15.40],
    ['May 25, 2023', 15.36],
    ['May 24, 2023', 15.49],
    ['May 23, 2023', 15.63],
    ['May 19, 2023', 15.68],
    ['May 18, 2023', 15.49],
    ['May 17, 2023', 15.68],
    ['May 16, 2023', 15.74],
    ['May 15, 2023', 15.96],
    ['May 12, 2023', 15.91],
    ['May 11, 2023', 15.95],
    ['May 10, 2023', 16.06],
    ['May 09, 2023', 16.10],
    ['May 08, 2023', 15.99],
    ['May 05, 2023', 15.96],
    ['May 04, 2023', 16.23],
    ['May 03, 2023', 16.14],
    ['May 02, 2023', 15.96],
    ['May 01, 2023', 15.69],
    ['April 28, 2023', 15.75],
    ['April 27, 2023', 15.73],
    ['April 26, 2023', 15.74],
    ['April 25, 2023', 15.81],
    ['April 24, 2023', 15.75],
    ['April 21, 2023', 15.70],
    ['April 20, 2023', 15.87],
    ['April 19, 2023', 15.79],
    ['April 18, 2023', 15.88],
    ['April 17, 2023', 15.79],
    ['April 14, 2023', 15.87],
    ['April 13, 2023', 16.15],
    ['April 12, 2023', 15.95],
    ['April 11, 2023', 15.86],
    ['April 10, 2023', 15.77],
    ['April 07, 2023', 16.00],
    ['April 06, 2023', 15.90],
    ['April 05, 2023', 16.00],
    ['April 04, 2023', 16.00],
    ['April 03, 2023', 15.72],
    ['March 31, 2023', 15.59],
    ['March 30, 2023', 15.68],
    ['March 29, 2023', 15.56],
    ['March 28, 2023', 15.63],
    ['March 27, 2023', 15.50],
    ['March 24, 2023', 15.67],
    ['March 23, 2023', 15.79],
    ['March 22, 2023', 15.60],
    ['March 21, 2023', 15.37],
    ['March 20, 2023', 15.67],
    ['March 17, 2023', 15.76],
    ['March 16, 2023', 15.21],
    ['March 15, 2023', 15.20],
    ['March 14, 2023', 15.08],
    ['March 13, 2023', 15.16],
    ['March 10, 2023', 14.80],
    ['March 09, 2023', 14.51],
    ['March 08, 2023', 14.37],
    ['March 07, 2023', 14.37],
    ['March 06, 2023', 14.63],
    ['March 03, 2023', 14.71],
    ['March 02, 2023', 14.55],
    ['March 01, 2023', 14.55],
    ['February 28, 2023', 14.47],
    ['February 27, 2023', 14.40],
    ['February 24, 2023', 14.35],
    ['February 23, 2023', 14.44],
    ['February 22, 2023', 14.46],
    ['February 21, 2023', 14.54],
    ['February 20, 2023', 14.55],
    ['February 17, 2023', 14.60],
    ['February 16, 2023', 14.55],
    ['February 15, 2023', 14.55],
    ['February 14, 2023', 14.69],
    ['February 13, 2023', 14.69],
    ['February 10, 2023', 14.78],
    ['February 09, 2023', 14.75],
    ['February 08, 2023', 14.86],
    ['February 07, 2023', 14.84],
    ['February 06, 2023', 14.80],
    ['February 03, 2023', 14.78],
    ['February 02, 2023', 15.16],
    ['February 01, 2023', 15.46],
    ['January 31, 2023', 15.28],
    ['January 30, 2023', 15.24],
    ['January 27, 2023', 15.28],
    ['January 26, 2023', 15.29],
    ['January 25, 2023', 15.42],
    ['January 24, 2023', 15.36],
    ['January 23, 2023', 15.31],
    ['January 20, 2023', 15.27]
]

nav_data = {}

for entry in data:
    date_str, nav = entry
    date = datetime.datetime.strptime(date_str, "%B %d, %Y").date()
    nav_data[date] = nav


In [107]:
# add all of the new entries in the nav_data dictionary to the phys2 dataframe.
for date, nav in nav_data.items():
    phys2.loc[date] = nav

datetime.date(2023, 6, 5)

In [111]:
# Write a method to sort a list of datetime.date objects in ascending order.
def sort_dates(dates):
    return sorted(dates)

# Write a method that accepts a date and a list of dates as input and returns the previous day to the given date that exists in the list of dates.
def get_previous_date(date, dates):
    dates = sort_dates(dates)
    date_index = dates.index(date)
    last_date = dates[date_index - 1]

# Write a method that accepts a date and computes the number of days ago it was from today.
def get_days_ago(date):
    today = datetime.date.today()
    return (today - date).days

# Write a method that accepts a dataframe (indexed on the date), a min date, and a max data and returns the dataframe data between those dates inclusive.
def get_data_between_dates(df, min_date, max_date):
    return df.loc[min_date:max_date]

# Write a method that takes a symbol, starting date and ending date and returns an array of close prices for that date range using yfinance.
def get_close_prices(symbol, start_date, end_date):
    data = yf.download(symbol, start_date, end_date)
    return data['Close'].values


In [108]:
# remove nulls
phys2 = phys2.dropna()
# convert to dict
phys2_dict = phys2.to_dict()
print(phys2_dict)

{'NAV': {datetime.date(2023, 1, 19): 15.3157, datetime.date(2023, 1, 18): 15.0929, datetime.date(2023, 1, 17): 15.1294, datetime.date(2023, 1, 13): 15.2216, datetime.date(2023, 1, 12): 15.0383, datetime.date(2023, 1, 11): 14.8689, datetime.date(2023, 1, 10): 14.8797, datetime.date(2023, 1, 9): 14.8385, datetime.date(2023, 1, 6): 14.7904, datetime.date(2023, 1, 5): 14.5305, datetime.date(2023, 1, 4): 14.7025, datetime.date(2023, 1, 3): 14.5831, datetime.date(2022, 12, 30): 14.4612, datetime.date(2022, 12, 29): 14.3891, datetime.date(2022, 12, 28): 14.3056, datetime.date(2022, 12, 27): 14.3798, datetime.date(2022, 12, 23): 14.2577, datetime.date(2022, 12, 22): 14.2128, datetime.date(2022, 12, 21): 14.3864, datetime.date(2022, 12, 20): 14.4147, datetime.date(2022, 12, 19): 14.1745, datetime.date(2022, 12, 16): 14.2183, datetime.date(2022, 12, 15): 14.0898, datetime.date(2022, 12, 14): 14.3315, datetime.date(2022, 12, 13): 14.3593, datetime.date(2022, 12, 12): 14.1266, datetime.date(2022, 

In [121]:
# Get GLD daily close data going back to jan 1 2022
gld_hist_daily = yf.download('PHYS', '2022-01-01', '2023-06-05')

# Convert the date to dates only
gld_hist_daily.index = gld_hist_daily.index.date

gld_hist_daily = gld_hist_daily.dropna()
gld_hist_daily = gld_hist_daily.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close'])

# convert closes to two decimal places
gld_hist_daily['Close'] = gld_hist_daily['Close'].apply(lambda x: round(x, 2))

# Convert the dataframe to a dictionary
gld_hist_daily_dict = gld_hist_daily.to_dict()
print(gld_hist_daily_dict)

[*********************100%***********************]  1 of 1 completed
{'Close': {datetime.date(2022, 1, 3): 14.13, datetime.date(2022, 1, 4): 14.24, datetime.date(2022, 1, 5): 14.19, datetime.date(2022, 1, 6): 14.05, datetime.date(2022, 1, 7): 14.09, datetime.date(2022, 1, 10): 14.12, datetime.date(2022, 1, 11): 14.3, datetime.date(2022, 1, 12): 14.36, datetime.date(2022, 1, 13): 14.31, datetime.date(2022, 1, 14): 14.29, datetime.date(2022, 1, 18): 14.27, datetime.date(2022, 1, 19): 14.48, datetime.date(2022, 1, 20): 14.49, datetime.date(2022, 1, 21): 14.51, datetime.date(2022, 1, 24): 14.66, datetime.date(2022, 1, 25): 14.67, datetime.date(2022, 1, 26): 14.46, datetime.date(2022, 1, 27): 14.29, datetime.date(2022, 1, 28): 14.25, datetime.date(2022, 1, 31): 14.3, datetime.date(2022, 2, 1): 14.32, datetime.date(2022, 2, 2): 14.35, datetime.date(2022, 2, 3): 14.31, datetime.date(2022, 2, 4): 14.32, datetime.date(2022, 2, 7): 14.42, datetime.date(2022, 2, 8): 14.45, datetime.date(2022, 2, 

In [22]:
# Testing reading it back in
phys2=pd.DataFrame.from_dict(phys2_dict)
# make the index titled Date
phys2.index.name='Date'
phys2.head()

,NAV,Close
Date,,
2017-12-29,10.6663,10.59
2017-12-28,10.6013,10.50
2017-12-27,10.5389,10.46
2017-12-26,10.5040,10.44
2017-12-22,10.4409,10.37


In [91]:
gld_daily = yf.download("GLD", start=min(phys2.index), end=max(phys2.index), interval="1d")
gld_daily.index = pd.to_datetime(gld_daily.index).date
gld_daily.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2018-01-02,124.660004,125.180000,124.389999,125.150002,125.150002,11762500
2018-01-03,125.050003,125.089996,124.099998,124.820000,124.820000,7904300
2018-01-04,124.889999,125.849998,124.739998,125.459999,125.459999,7329700
2018-01-05,124.930000,125.480003,124.830002,125.330002,125.330002,5739900
2018-01-08,125.199997,125.320000,124.900002,125.309998,125.309998,3566700


In [99]:
from datetime import datetime
date = datetime.strptime('2023-01-10', '%Y-%m-%d').date()
pnav_ratio = list(phys2.loc[:date].iloc[-10:]['Close'] / phys2.loc[:date].iloc[-10:]['NAV'])
np.mean(pnav_ratio)

0.9775853608189348

In [100]:
import datetime

data = [
    ['June 05, 2023', 15.52],
    ['June 02, 2023', 15.41],
    ['June 01, 2023', 15.65],
    ['May 31, 2023', 15.53],
    ['May 30, 2023', 15.50],
    ['May 26, 2023', 15.40],
    ['May 25, 2023', 15.36],
    ['May 24, 2023', 15.49],
    ['May 23, 2023', 15.63],
    ['May 19, 2023', 15.68],
    ['May 18, 2023', 15.49],
    ['May 17, 2023', 15.68],
    ['May 16, 2023', 15.74],
    ['May 15, 2023', 15.96],
    ['May 12, 2023', 15.91],
    ['May 11, 2023', 15.95],
    ['May 10, 2023', 16.06],
    ['May 09, 2023', 16.10],
    ['May 08, 2023', 15.99],
    ['May 05, 2023', 15.96],
    ['May 04, 2023', 16.23],
    ['May 03, 2023', 16.14],
    ['May 02, 2023', 15.96],
    ['May 01, 2023', 15.69],
    ['April 28, 2023', 15.75],
    ['April 27, 2023', 15.73],
    ['April 26, 2023', 15.74],
    ['April 25, 2023', 15.81],
    ['April 24, 2023', 15.75],
    ['April 21, 2023', 15.70],
    ['April 20, 2023', 15.87],
    ['April 19, 2023', 15.79],
    ['April 18, 2023', 15.88],
    ['April 17, 2023', 15.79],
    ['April 14, 2023', 15.87],
    ['April 13, 2023', 16.15],
    ['April 12, 2023', 15.95],
    ['April 11, 2023', 15.86],
    ['April 10, 2023', 15.77],
    ['April 07, 2023', 16.00],
    ['April 06, 2023', 15.90],
    ['April 05, 2023', 16.00],
    ['April 04, 2023', 16.00],
    ['April 03, 2023', 15.72],
    ['March 31, 2023', 15.59],
    ['March 30, 2023', 15.68],
    ['March 29, 2023', 15.56],
    ['March 28, 2023', 15.63],
    ['March 27, 2023', 15.50],
    ['March 24, 2023', 15.67],
    ['March 23, 2023', 15.79],
    ['March 22, 2023', 15.60],
    ['March 21, 2023', 15.37],
    ['March 20, 2023', 15.67],
    ['March 17, 2023', 15.76],
    ['March 16, 2023', 15.21],
    ['March 15, 2023', 15.20],
    ['March 14, 2023', 15.08],
    ['March 13, 2023', 15.16],
    ['March 10, 2023', 14.80],
    ['March 09, 2023', 14.51],
    ['March 08, 2023', 14.37],
    ['March 07, 2023', 14.37],
    ['March 06, 2023', 14.63],
    ['March 03, 2023', 14.71],
    ['March 02, 2023', 14.55],
    ['March 01, 2023', 14.55],
    ['February 28, 2023', 14.47],
    ['February 27, 2023', 14.40],
    ['February 24, 2023', 14.35],
    ['February 23, 2023', 14.44],
    ['February 22, 2023', 14.46],
    ['February 21, 2023', 14.54],
    ['February 20, 2023', 14.55],
    ['February 17, 2023', 14.60],
    ['February 16, 2023', 14.55],
    ['February 15, 2023', 14.55],
    ['February 14, 2023', 14.69],
    ['February 13, 2023', 14.69],
    ['February 10, 2023', 14.78],
    ['February 09, 2023', 14.75],
    ['February 08, 2023', 14.86],
    ['February 07, 2023', 14.84],
    ['February 06, 2023', 14.80],
    ['February 03, 2023', 14.78],
    ['February 02, 2023', 15.16],
    ['February 01, 2023', 15.46],
    ['January 31, 2023', 15.28],
    ['January 30, 2023', 15.24],
    ['January 27, 2023', 15.28],
    ['January 26, 2023', 15.29],
    ['January 25, 2023', 15.42],
    ['January 24, 2023', 15.36],
    ['January 23, 2023', 15.31],
    ['January 20, 2023', 15.27],
    ['January 19, 2023', 15.32],
    ['January 18, 2023', 15.09],
    ['January 17, 2023', 15.13],
    ['January 16, 2023', 15.04],
    ['January 13, 2023', 15.22]
]

nav_data = {}

for entry in data:
    date_str, nav = entry
    date = datetime.datetime.strptime(date_str, "%B %d, %Y").date()
    nav_data[date] = nav

print(nav_data)


{datetime.date(2023, 6, 5): 15.52, datetime.date(2023, 6, 2): 15.41, datetime.date(2023, 6, 1): 15.65, datetime.date(2023, 5, 31): 15.53, datetime.date(2023, 5, 30): 15.5, datetime.date(2023, 5, 26): 15.4, datetime.date(2023, 5, 25): 15.36, datetime.date(2023, 5, 24): 15.49, datetime.date(2023, 5, 23): 15.63, datetime.date(2023, 5, 19): 15.68, datetime.date(2023, 5, 18): 15.49, datetime.date(2023, 5, 17): 15.68, datetime.date(2023, 5, 16): 15.74, datetime.date(2023, 5, 15): 15.96, datetime.date(2023, 5, 12): 15.91, datetime.date(2023, 5, 11): 15.95, datetime.date(2023, 5, 10): 16.06, datetime.date(2023, 5, 9): 16.1, datetime.date(2023, 5, 8): 15.99, datetime.date(2023, 5, 5): 15.96, datetime.date(2023, 5, 4): 16.23, datetime.date(2023, 5, 3): 16.14, datetime.date(2023, 5, 2): 15.96, datetime.date(2023, 5, 1): 15.69, datetime.date(2023, 4, 28): 15.75, datetime.date(2023, 4, 27): 15.73, datetime.date(2023, 4, 26): 15.74, datetime.date(2023, 4, 25): 15.81, datetime.date(2023, 4, 24): 15.7